In [ ]:
!pip install pyodbc # ODBC的DRIVER(驅動程式)

### ----開始執行----

In [1]:
# coding=utf-8
from platform import python_version
import os, time, socket, glob, pickle, joblib
import pandas as pd
import pyodbc, pymssql
import warnings
warnings.filterwarnings(action="ignore")
print("【現在時間】{}".format(time.strftime("%Y/%m/%d %H:%M:%S")))
print("【所在目錄】{}".format(os.getcwd()))
#print("【主機名稱】{} ({})".format(socket.gethostname(),socket.gethostbyname(socket.
print("【Python】{}".format(python_version()))
print("【pymssql】{}".format(pymssql.__version__))
print("【pyodbc】{}".format(pyodbc.version))
%autosave 120

【現在時間】2022/12/18 17:04:04
【所在目錄】C:\Users\User\Desktop\資料工程
【Python】3.9.13
【pymssql】2.2.7
【pyodbc】4.0.35


Autosaving every 120 seconds


In [2]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

## 董監持股

In [18]:
stock_id = "1218"  #1605 1218 6446
url = 'https://goodinfo.tw/tw/StockDirectorSharehold.asp?STOCK_ID='+stock_id
payloads = {"STOCK_ID":stock_id}
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
Chrome/107.0.0.0 Safari/537.36"}
resp = requests.request(method="POST",url=url,data=payloads,headers=headers)
resp.encoding = "utf-8"

table_s = pd.read_html(resp.text)


In [19]:
df = table_s[15].dropna(axis='columns')
print(len(df))
df.head(2)

38


月別  當月股價               發行張數(萬張) 非獨立董監持股                           ...  \
        月別  當月收盤  漲跌(元)  漲跌(%) 發行張數(萬張)    持股張數 持股(%) 持股增減    質押張數 質押(%)  ...   
0  2022/12  36.5  -10.5  -22.3       50       -     -    -       -     -  ...   
1  2022/11    47  +8.05  +20.7       50  182312  36.5    0  147123  80.7  ...   

  獨立董監持股                  全體董監持股                          外資持股(%)  
   持股(%) 持股增減 質押張數 質押(%)    持股張數 持股(%) 持股增減    質押張數 質押(%) 外資持股(%)  
0      -    -    -     -       -     -    -       -     -     2.6  
1      0    0    0     0  182312  36.5    0  147123  80.7     3.4  

[2 rows x 21 columns]

In [20]:
products_list = df.values.tolist()
print(products_list[19])

del products_list[19]
del products_list[18]
lists_num=len(products_list)
print(len(products_list[1]))
print(products_list[1])
"0 1   4 5 10   20"

['月別', '當月收盤', '漲跌(元)', '漲跌(%)', '發行張數(萬張)', '持股張數', '持股(%)', '持股增減', '質押張數', '質押(%)', '持股張數', '持股(%)', '持股增減', '質押張數', '質押(%)', '持股張數', '持股(%)', '持股增減', '質押張數', '質押(%)', '外資持股(%)']
21
['2022/11', '47', '+8.05', '+20.7', '50', '182312', '36.5', '0', '147123', '80.7', '0', '0', '0', '0', '0', '182312', '36.5', '0', '147123', '80.7', '3.4']


'0 1   4 5 10   20'

### SQL連線

In [6]:
connsqlodbc= pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};SERVER=124.218.194.84;"
                      "DATABASE=testDB;UID=testuser;PWD=323")


In [7]:
connsqlodbc.autocommit = True
cursor = connsqlodbc.cursor()
# GS+代號+發周別
# MYSQL指令
time_S=time.strftime("%Y/%m/%d")
print(time_S)
sqlStr ='''create table 董監持股比例表
(
	董監持股比例表編號 nvarchar(20) primary key
    ,[當月收盤] nvarchar(20) Null
	,[發行張數] nvarchar(20) Null
	,[非獨立董監持股] nvarchar(20) NULL
	,[獨立董監持股] nvarchar(20) Null
	,[外資持股(%)] nvarchar(20) Null
	,董監持股月別 nvarchar(10) not Null
	,股票代號 nvarchar(10) not Null
)
'''
cursor.execute(sqlStr)

2022/12/18


In [21]:
connsqlodbc.autocommit = True

connsqlodbc.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
connsqlodbc.setencoding('utf-8')
cursor = connsqlodbc.cursor()
# P+代號+發周別
# MYSQL指令
for i in range(0,lists_num):
    Pstock_id='P'+str(stock_id)+str(products_list[i][0])
    sqlStr = f""" 
   Insert into 董監持股比例表 (董監持股比例表編號
    ,[當月收盤]
	,[發行張數],[非獨立董監持股]
	,[獨立董監持股]
	,[外資持股(%)]
    ,董監持股月別,股票代號)
    values ('{str(Pstock_id)}',
    '{products_list[i][1]}','{products_list[i][4]}','{products_list[i][5]}','{products_list[i][10]}',
    '{products_list[i][20]}',
    '{products_list[i][0]}','{stock_id}')
    
    """
    cursor.execute(sqlStr)

print(sqlStr)



 
   Insert into 董監持股比例表 (董監持股比例表編號
    ,[當月收盤]
	,[發行張數],[非獨立董監持股]
	,[獨立董監持股]
	,[外資持股(%)]
    ,董監持股月別,股票代號)
    values ('P12182020/01',
    '19.95','50','10190','0',
    '2.3',
    '2020/01','1218')
    
    


In [22]:
connsqlodbc.close()